In [11]:
import boto3
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'ryan-greiner-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'predictive_analytics/Iris.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading CSV file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [12]:
## Feature Engineering ##
iris['species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1,
                                np.where(iris['Species'] == 'Iris-versicolor', 2, 3))

## Define variables ##
X = iris.drop(columns = ['Id', 'Species', 'species_numb'], axis = 1)
Y = iris['species_numb']

## Split Data ##
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)

## Scale Data ##
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Logistic Regression

In [13]:
## Model ##
log_md = OneVsOneClassifier(estimator = LogisticRegression()).fit(X_train, Y_train)

## Predictions ##
log_pred = log_md.predict(X_test)
print(confusion_matrix(Y_test, log_pred))
print(classification_report(Y_test, log_pred))

[[10  0  0]
 [ 1  9  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.91      1.00      0.95        10
           2       1.00      0.90      0.95        10
           3       1.00      1.00      1.00        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



This model mistakenly predicted a 2 as a 1

# Decision Tree

In [14]:
## Model ##
tree_md = OneVsOneClassifier(estimator = DecisionTreeClassifier(max_depth = 3)).fit(X_train, Y_train)

## Predictions ##
tree_pred = tree_md.predict(X_test)
print(confusion_matrix(Y_test, tree_pred))
print(classification_report(Y_test, tree_pred))

[[ 9  1  0]
 [ 1  9  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



This model miss